## Chest Abnormality Detection with YOLO-v3

![yolo](https://miro.medium.com/max/3802/1*d4Eg17IVJ0L41e7CTWLLSg.png)

## Credits

Please go and visit these below mentioned notebooks and support their works. Without the help of the below mentioned notebooks, it would have been much difficult for me to approach the solution.

1. [VinBigData-CXR-AD YOLOv5 14 Class [train]](https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-train)

2. [VinBigData-CXR-AD YOLOv5 14 Class [infer]](https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-infer)

3. [VinBigData: Process and resize to PNG (1024x1024)](https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-png-1024x1024)

4. [VinBigData: Process and resize to image](https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image)

5. [chest generate training folds](https://www.kaggle.com/abhishek/chest-generate-training-folds)

6. video -> [Train custom object detection model with YOLO V5](https://youtu.be/NU9Xr_NYslo) by [Abhishek Thakur](https://www.kaggle.com/abhishek)

## My Previous work on VinBigData Chest X-ray Abnormalities Detection

1. [Chest X-ray Abnormalities Detection](https://www.kaggle.com/basu369victor/chest-x-ray-abnormalities-detection)

Topics utilized in the above notebook.
   * FasterRCNN from torchvision
   * Uses Resnet50+FPN backbone
   * Normalization of the X-ray images
   * Visualization of Model performance over validation images

2. [Chest X-ray Abnormalities Detection(Submission)](https://www.kaggle.com/basu369victor/chest-x-ray-abnormalities-detection-submission)


**If you find my previous notebooks informative and useful please do support my work by upvoting them and also let me know your opinions in the comments**.

In [ ]:
!pip install bbox-visualizer

In [ ]:
import numpy as np
import pandas as pd
from glob import glob
import shutil, os

import matplotlib.pyplot as plt
import seaborn as sns
import bbox_visualizer as bbv

from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm

import cv2

In [ ]:
size = 512
BASE_DIR = '../input/vinbigdata-chest-xray-abnormalities-detection/'
if size == 512:
    External_DIR = '../input/vinbigdata'
if size == 1024:
    External_DIR = '../input/vinbigdata-chest-xray-resized-png-1024x1024'

In [ ]:
train_df = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))
train_df.head()

In [ ]:
train_df = train_df[train_df.class_name!='No finding'].reset_index(drop=True)

In [ ]:
train_dim = pd.read_csv(os.path.join(External_DIR, "train_meta.csv"))
train_dim.head()

In [ ]:
train = pd.merge(train_df, train_dim, on='image_id')
train.head()

## Pre-processing

In [ ]:
# reshaping the bounding-box w.r.t. the resized image
train['x_min'] = train.apply(lambda row: (row.x_min)/row.dim1, axis = 1)*float(size)
train['y_min'] = train.apply(lambda row: (row.y_min)/row.dim0, axis = 1)*float(size)

train['x_max'] = train.apply(lambda row: (row.x_max)/row.dim1, axis =1)*float(size)
train['y_max'] = train.apply(lambda row: (row.y_max)/row.dim0, axis =1)*float(size)

# calculation x-mid, y-mid, width and hight of the bounding box for yolo
train['x_mid'] = train.apply(lambda row: (row.x_max+row.x_min)/2, axis =1)
train['y_mid'] = train.apply(lambda row: (row.y_max+row.y_min)/2, axis =1)

train['w'] = train.apply(lambda row: (row.x_max-row.x_min), axis =1)
train['h'] = train.apply(lambda row: (row.y_max-row.y_min), axis =1)

train['x_mid'] /= float(size)
train['y_mid'] /= float(size)

train['w'] /= float(size)
train['h'] /= float(size)

train['area'] = train['w']*train['h']
train.head()

In [ ]:
Kfold  = GroupKFold(n_splits = 3)
train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(Kfold.split(train, groups = train.image_id.values)):
    train.loc[val_idx, 'fold'] = fold
train.head()

In [ ]:
fold_0 = 1
fold_1 = 2
train_files = []
val_files   = []

train_files += list(train[train.fold==fold_0].image_id.unique())
val_files += list(train[train.fold==fold_1].image_id.unique())
len(train_files), len(val_files)

## Chest X-ray bounding bounding box visualization

If the image size is 1024, you could use the **visualize_plot** function to visualize x-rays with bounding boxes, but I would not recommend this in case of 512 or lower size images because of the low-quality plots in the final visualized result.<br><br>
I have already created similar plots in my [Chest X-ray Abnormalities Detection](https://www.kaggle.com/basu369victor/chest-x-ray-abnormalities-detection) notebook, so you could go and check them out.


In [ ]:
def visualize_plot(idx):
    image = train_files[idx]
    records = train[train['image_id'] == image]
    boxes = np.array(records[['x_min','y_min','x_max','y_max']])
    
    labels = records.class_name
    sample = cv2.imread(os.path.join('/content','train',f'{image}.png'))
    img = sample.copy()
    plt.figure(figsize=(16, 16))
    for box,label in zip(boxes,labels):
        bbv.add_label(img, 
                      label, 
                      [int(round(box[0])), int(round(box[1])),int(round(box[2])), int(round(box[3]))], 
                      draw_bg=True,
                      text_bg_color=(255,0,0),
                      text_color=(0,0,0),
                      )
        cv2.rectangle(img ,
                      (int(round(box[0])), int(round(box[1]))),
                      (int(round(box[2])), int(round(box[3]))),
                      (255,0,0),
                      2)


    plt.imshow(img)

In [ ]:
#visualize_plot(0)

In [ ]:
#visualize_plot(12)

In [ ]:
#visualize_plot(24)

In [ ]:
#visualize_plot(30)

In [ ]:
#visualize_plot(62)

## Directory Tree for YOLO

In [ ]:
os.makedirs('./vinbigdata/labels/train', exist_ok = True)
os.makedirs('./vinbigdata/labels/val', exist_ok = True)
os.makedirs('./vinbigdata/images/train', exist_ok = True)
os.makedirs('./vinbigdata/images/val', exist_ok = True)

In [ ]:
TRAIN_LABELS_PATH = './vinbigdata/labels/train'
VAL_LABELS_PATH = './vinbigdata/labels/val'
TRAIN_IMAGES_PATH = './vinbigdata/images/train'
VAL_IMAGES_PATH = './vinbigdata/images/val'

for file in tqdm(train_files):
    records = train[train['image_id'] == file]
    attributes = records[['class_id','x_mid','y_mid','w','h']].values
    attributes = np.array(attributes)
    np.savetxt(
        os.path.join(
            TRAIN_LABELS_PATH,
            f"{file}.txt"
        ),
        attributes,
        fmt = ["%d","%f","%f","%f","%f"]
    )
    shutil.copy(
        os.path.join(
            External_DIR,
            'train',
            f"{file}.png" 
        ),          
        TRAIN_IMAGES_PATH
    )

In [ ]:
for file in tqdm(val_files):
    records = train[train['image_id'] == file]
    attributes = records[['class_id','x_mid','y_mid','w','h']]
    attributes = np.array(attributes)
    np.savetxt(
        os.path.join(
            VAL_LABELS_PATH,
            f"{file}.txt"
        ),
        attributes,
        fmt = ["%d","%f","%f","%f","%f"]
    )
    shutil.copy(
        os.path.join(
            External_DIR,
            'train',
            f"{file}.png" 
        ),          
        VAL_IMAGES_PATH
    )

In [ ]:
class_ids, class_names = list(zip(*set(zip(train.class_id, train.class_name))))
classes = list(np.array(class_names)[np.argsort(class_ids)])
classes = list(map(lambda x: str(x), classes))
classes

In [ ]:
from os import listdir
from os.path import isfile, join
import yaml

cwd = './'

with open(join( cwd , 'train.txt'), 'w') as f:
    for path in glob('./vinbigdata/images/train/*'):
        f.write(path+'\n')
            
with open(join( cwd , 'val.txt'), 'w') as f:
    for path in glob('./vinbigdata/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  '../train.txt',
    val   =  '../val.txt',
    nc    = 14,
    names = classes
    )

with open(join( cwd , 'vinbigdata.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
    
f = open(join( cwd, 'vinbigdata.yaml'), 'r')
print('\nyaml:')
print(f.read())

## Explanation

So I was trying to approach this problem with [**Scaled YOLO-v4**](https://arxiv.org/pdf/2011.08036.pdf), but the approach did not go well, the recall score was pretty low and the model was not predicting the bounding box. My assumptions are that if the model was trained for many iterations or if the model parameters would have been tuned manually like iou_threshold, or convolution layer parameters the model would have worked fine.<br><br>

Now, Yolo-v5 and Yolo-v4 are far better than Yolo-v3 then why did I go for Yolo-v3. Yolo-v5 for this problem has already been implemented in this notebook- [VinBigData-CXR-AD YOLOv5 14 Class [train]](https://www.kaggle.com/awsaf49/vinbigdata-cxr-ad-yolov5-14-class-train)
, and I think the performance is really good. For Yolo-v4, I kept it for later experimentation. And with Yolo-v3 I tried because I was curious to know how an older methodology would work over a medical imaging scenario. 

Also, I came across this research paper where YOLO-v2 with DenseNet201 in backend network has been used for chest x-ray abnormality detection, [Reproducibility of abnormality detection on chest radiographs using convolutional neural network in paired radiographs obtained within a short-term interval](https://www.nature.com/articles/s41598-020-74626-4). I think this research paper is pretty interesting and you could give it a look. The output layers of eDenseYOLO, which is You Only Look Once v2 with DenseNet201, were modified for improved robustness to the variable size of disease patterns. If the input resolution was 256 × 256, the feature map for the last layer was 8 × 8, 16 × 16, and 32 × 32 with skip connection.

## Implementation of Scaled YOLO-v4


I have commented out the scaled Yolo implementation part because the final results were not promising, but you could give it a try in your side, and have promising and better solution over this problem.

In [ ]:
#!git clone https://github.com/AlexeyAB/darknet
#!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-csp.weights
#!git clone -b yolov4-csp https://github.com/WongKinYiu/ScaledYOLOv4.git

In [ ]:
#!git clone https://github.com/JunnYu/mish-cuda.git
#%cd mish-cuda
#!python setup.py build install
#%cd ..

In [ ]:
#!git clone https://github.com/WongKinYiu/ScaledYOLOv4.git
#%cd ./ScaledYOLOv4/
#!git checkout yolov4-csp

In [ ]:
#!python train.py --img 512 512 --batch-size 24 --epochs 20 --data ../vinbigdata.yaml --cfg yolov4-csp.cfg --weights ../yolov4-csp.weights --cache

## Yolo V3

In [ ]:
!git clone https://github.com/ultralytics/yolov3.git
%cd ./yolov3/
!pip install -r requirements.txt

In [ ]:
!wget https://github.com/ultralytics/yolov3/releases/download/v9.1/yolov3.pt
#!wget https://github.com/ultralytics/yolov3/releases/download/v9.1/yolov3-spp.pt
#!wget https://github.com/ultralytics/yolov3/releases/download/v9.1/yolov3-tiny.pt

In [ ]:
!WANDB_MODE="dryrun" python train.py --img {size} --batch-size 40 --epochs 60 --data ../vinbigdata.yaml --weights yolov3.pt

## Yolo performance evaluation

In [ ]:
!pip install matplotlib==3.1.3
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/results.png'));

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('runs/train/exp/precision_recall_curve.png'));

In [ ]:
plt.figure(figsize=(30,15))
plt.axis('off')
plt.imshow(plt.imread('./runs/train/exp/labels.jpg'));

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 20))

ax[0].imshow(plt.imread('runs/train/exp/test_batch0_labels.jpg'))
ax[1].imshow(plt.imread('runs/train/exp/test_batch0_pred.jpg'))
ax[0].title.set_text('Ground Truth')
ax[1].title.set_text('YOLO predictions')

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 20))

ax[0].imshow(plt.imread('runs/train/exp/test_batch1_labels.jpg'))
ax[1].imshow(plt.imread('runs/train/exp/test_batch1_pred.jpg'))
ax[0].title.set_text('Ground Truth')
ax[1].title.set_text('YOLO predictions')

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 20))

ax[0].imshow(plt.imread('runs/train/exp/test_batch2_labels.jpg'))
ax[1].imshow(plt.imread('runs/train/exp/test_batch2_pred.jpg'))
ax[0].title.set_text('Ground Truth')
ax[1].title.set_text('YOLO predictions')

In [ ]:
for file in (glob('runs/train/exp/**/*.png', recursive = True)+glob('runs/train/exp/**/*.jpg', recursive = True)):
    os.remove(file)

In [ ]:
%cd ..
shutil.rmtree('vinbigdata')

## The End

The Inference notebook would be availale at [Chest X-ray Abnormality Detection YOLOv3 [Infer]](https://www.kaggle.com/basu369victor/chest-x-ray-abnormality-detection-yolov3-infer)